# Mini Project 1

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

**Instructions** Use current stock data to create two potentially profitable investment portfolios. One that is higher risk and one that is lower risk.

-- You are to interpret and explain your interpretation of a high risk profile and low risk profile of a portfolio. You should provide some measurable quantitative data in your explanation.

## Import Packages

In [102]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.optimize import minimize
import datetime as dt
sns.set_style('darkgrid')

import yfinance as yf
import bs4 as bs
import requests
import random

## High Risk Portfolio

For my high risk portfolio, I selected stocks in the technology sector that had a value of less than $5 per share. My thinking was technology seems like a more volitile sector, and low cost stocks are more likely to be volitile - like penny stocks. I randomly selected 10 stocks under these criteria. They are listed below in the ticker list.

In [103]:
# List of tickers for the stocks to be analyzed
tickers_high = ['WIT', 'DIDIY', 'ZTCOF', 'GRAB', 'LNVGF', 'BYDIF', 'HHUSF', 'HPHTY', 'BB', 'AWEVF']

# Set the date range for the past two years
start_date = dt.datetime.today()-dt.timedelta(days = 2*365)
end_date = dt.datetime.today()

# Download stock data
stock = yf.download(tickers_high, start = start_date, end = end_date)

[*********************100%***********************]  10 of 10 completed


In [104]:
# Create a dataframe of daily returns - log is taken to reduce computational complexity
daily_returns = np.log(stock['Close']/stock['Close'].shift(1))
daily_returns = daily_returns.dropna()


# Create a dictionary of standard deviations of daily return
# Normalize the standard deviation for yearly by multiplying by sqrt(number of trading days in year ~ 252)
annualized_volatility_high = {ticker: np.std(daily_returns[ticker])*np.sqrt(252) for ticker in tickers_high}

In [105]:
# Calculate the covariance matrix
covariance_matrix_high = 252*((daily_returns).cov())

In [106]:
# Use scipy minimize to find the optimal weights for the portfolio to minimize volatility
# Constraints for the portfolio weights:
    # 1) Sum of weights equals 1
    # 2) No weight can be less than 0
    # 3) No weight can be greater than 0.25

# Number of assets
n_assets = len(tickers_high)

# Define an initial guess for asset weights as equal distribution
initial_weights = np.array([1/n_assets] * n_assets)

# Define weight constraints listed above
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1},
               {'type': 'ineq', 'fun': lambda weights: min(weights)},
              {'type': 'ineq', 'fun': lambda weights: .25-max(weights)})

# Define the objective function to minimize portfolio variance
def portfolio_volatility(weights):
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix_high, weights)))
    return portfolio_std_dev

# Run the optimization to find the optimal weights
result_high = minimize(portfolio_volatility, initial_weights, constraints=constraints)

# Optimal asset weights
optimal_weights_high = result_high.x

## Low Risk Portfolio

For my low risk portfolio, I will randomly select 10 stocks from the S&P500 to use. My thought process was that this should typically follow market trends and have a relatively low volatility.

In [107]:
# Get the list of S&P 500 tickers from Wikipedia
# Get the web data from the Wikipedia page
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# Parse the HTML content using BeautifulSoup
soup = bs.BeautifulSoup(resp.text, 'lxml') 
# Find the table containing the S&P 500 tickers_high
table = soup.find('table', {'class': 'wikitable sortable sticky-header', 'id': 'constituents'})
tickers = []

# Extract the tickers from the table
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

# Clean the tickers
tickers = [s.replace('\n', '') for s in tickers]
tickers = [s.replace('.', '-') for s in tickers]

# Randomly select 10 tickers from the S&P 500 list
rand_tickers = random.sample(tickers, 10)

In [108]:
# Set the date range for the past two years
start_date = dt.datetime.today()-dt.timedelta(days = 2*365)
end_date = dt.datetime.today()

# Download stock data
stock = yf.download(rand_tickers, start = start_date, end = end_date)

[*********************100%***********************]  10 of 10 completed


In [109]:
# Create a dataframe of daily returns - log is taken to reduce computational complexity
daily_returns = np.log(stock['Close']/stock['Close'].shift(1))
daily_returns = daily_returns.dropna()


# Create a dictionary of standard deviations of daily return
# Normalize the standard deviation for yearly by multiplying by sqrt(number of trading days in year ~ 252)
annualized_volatility_low = {ticker: np.std(daily_returns[ticker])*np.sqrt(252) for ticker in rand_tickers}

In [110]:
# Calculate the covariance matrix
covariance_matrix_low = 252*((daily_returns).cov())

In [111]:
# Use scipy minimize to find the optimal weights for the portfolio to minimize volatility
# Constraints for the portfolio weights:
    # 1) Sum of weights equals 1
    # 2) No weight can be less than 0
    # 3) No weight can be greater than 0.25

# Number of assets
n_assets = len(rand_tickers)

# Define an initial guess for asset weights as equal distribution
initial_weights = np.array([1/n_assets] * n_assets)

# Define weight constraints listed above
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1},
               {'type': 'ineq', 'fun': lambda weights: min(weights)},
              {'type': 'ineq', 'fun': lambda weights: .25-max(weights)})

# Define the objective function to minimize portfolio variance
def portfolio_volatility(weights):
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix_low, weights)))
    return portfolio_std_dev

# Run the optimization to find the optimal weights
result_low = minimize(portfolio_volatility, initial_weights, constraints=constraints)

# Optimal asset weights
optimal_weights = result_low.x

## Portfolio Comparison

First, let's see the portfolio volitility comparison

In [112]:
print(f' The optimal volatility subject to the constraints for the low risk portfolio is {result_low.fun}')
print(f' The optimal volatility subject to the constraints for the high risk portfolio is {result_high.fun}')

 The optimal volatility subject to the constraints for the low risk portfolio is 0.14758082060166325
 The optimal volatility subject to the constraints for the high risk portfolio is 0.2503089185134767


As we see above, the low risk portfolio is much less volitile than the high risk portfolio. This makes sense when you look at the annulized volatility of the two sets of stocks. The individual volatilities for the high risk stocks are much higher than those of the low risk stocks.

In [113]:
print(f'The annualized volatility for the low risk stocks is')
for stock in annualized_volatility_low: print(f'{stock}: {annualized_volatility_low[stock]:.4f}')
print('')
print(f'The annualized volatility for the high risk stocks is')
for stock in annualized_volatility_high: print(f'{stock}: {annualized_volatility_high[stock]:.4f}')

The annualized volatility for the low risk stocks is
BALL: 0.2732
AZO: 0.2125
AKAM: 0.3467
DVA: 0.3370
DG: 0.4598
IQV: 0.3063
LVS: 0.3315
HUM: 0.3950
TMO: 0.2439
VTR: 0.2426

The annualized volatility for the high risk stocks is
WIT: 0.2836
DIDIY: 0.5078
ZTCOF: 1.0403
GRAB: 0.5510
LNVGF: 0.9141
BYDIF: 0.7793
HHUSF: 1.0892
HPHTY: 0.7202
BB: 0.6289
AWEVF: 0.9430


The weights our optimizer selected for each portfolio are listed below:

In [114]:
print("Optimal weights for low volatility profile:")
for ticker, weight in zip(rand_tickers, optimal_weights):
    print(f"{ticker}: Weight = {weight:.4f}")
print('')
print("Optimal weights for high volatility profile:")
for ticker, weight in zip(tickers_high, optimal_weights_high):
    print(f"{ticker}: Weight = {weight:.4f}")

Optimal weights for low volatility profile:
BALL: Weight = 0.0663
AZO: Weight = 0.2500
AKAM: Weight = 0.0719
DVA: Weight = 0.0559
DG: Weight = 0.0973
IQV: Weight = 0.0818
LVS: Weight = -0.0000
HUM: Weight = 0.0390
TMO: Weight = 0.1180
VTR: Weight = 0.2198

Optimal weights for high volatility profile:
WIT: Weight = 0.0466
DIDIY: Weight = 0.0909
ZTCOF: Weight = 0.0436
GRAB: Weight = 0.1281
LNVGF: Weight = 0.1664
BYDIF: Weight = 0.0396
HHUSF: Weight = 0.1825
HPHTY: Weight = 0.0171
BB: Weight = 0.2500
AWEVF: Weight = 0.0352
